In [17]:
import laion_clap
import numpy as np
import os
import csv
import torch

Outline a simple testing loop, on TrueMedia data 
Then also try training it on in-the-wild data

In [6]:
CLAP_EMBEDDING_DIMENSION = 512

In [7]:
clap_model = laion_clap.CLAP_Module(enable_fusion=False)
clap_model.load_ckpt() # download the default pretrained checkpoint.

/home/ubuntu/.conda/envs/clap2/lib/python3.10/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755897462/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/ubuntu/.conda/envs/clap2/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on an

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

# Get the audio files

In [ ]:
from torch.utils.data import Dataset
from pydub import AudioSegment
import pandas as pd

import os

# create a dataloader for TrueMedia audio files 
# There should also be a dataloader for In-The-Wild data but idk where that is- fastest build my own 

def get_audio_length(path):
    audio = AudioSegment.from_file(path)
    duration_seconds = len(audio) / 1000.0
    return duration_seconds

class Dataset_CLAP_Embeddings(Dataset):
    def __init__(self, embeddings, labels):
        """self.list_filenames	: list of strings ,
           self.labels      : list of labels- binary 1 for Fake 0 for real"""
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, index):
        return self.embeddings[index], self.labels[index]


# takes in audio_media csv file and the dir to the audio media 


def get_clap_embeddings(model, audio_files):
    embedding_dimension = CLAP_EMBEDDING_DIMENSION
    embeddings = np.zeros(shape=(len(audio_files), embedding_dimension))
    error_count = 0
    for i, file in enumerate(audio_files): # cannot get all embeddings at once without running out of memory
        try:
            embeddings[i] = model.get_audio_embedding_from_filelist(x=[file], use_tensor=False)
        except Exception as e:
            print(f"Error processing file: {e}")
            error_count+=1
            embeddings[i] = None
    print("Number of embedding errors", error_count)
    return embeddings

def filter_tm_audio_data(audio_media_csv, audio_media_dir, min_duration):
    """Filters out unknown labeled data, data without a proper file path and converts ground truth to 1 for fake 0 for real
    returns list of the full filepaths and labels
    """
    df = pd.read_csv(audio_media_csv)
    filepaths = []
    labels = []
    count_unknown=0
    for _, row in df.iterrows():
        item_id = row['Id']
        item_audio_id = row['Audio ID']
        item_id_path = f'{audio_media_dir}/{item_id}'
        item_audio_id_path = f'{audio_media_dir}/{item_audio_id}'
        if os.path.isfile(item_id_path):
            item = item_id
        elif os.path.isfile(item_audio_id_path):
            item = item_audio_id
        else:
            print(f"Error: Neither Id '{item_id}' nor Audio ID '{item_audio_id}' found in directory")
            continue

        label = row['Ground Truth'] # Pull ground truth label for the current audio track
        if label == "Unknown":
            count_unknown += 1
            continue

        path = f'{audio_media_dir}/{item}'

        try:
            duration = get_audio_length(path)
            if duration < min_duration:
                filtered += 1
                continue
        except Exception as e:
            print(id, e)
    
        # Change label to be 1 and 0
        if label.lower() == 'fake':
            labels.append(1)
        elif label.lower() == 'real':
            labels.append(0)
        else:
            print("ERROR unexpected label:", label)
            assert False
        filepaths.append(path)
    
    print('Total count unknown', count_unknown)
    return filepaths, labels

def get_tm_audio_dataset(embedding_model, audio_media_csv, audio_media_dir):
    # get a list of filepaths 
    # get a list of corresponding labels
    filepaths, labels = filter_tm_audio_data(audio_media_csv=audio_media_csv, audio_media_dir=audio_media_dir, min_duration=4)
    embeddings = get_clap_embeddings(model=embedding_model, audio_files=filepaths)
    # mask = [embedding is not None for embedding in embeddings]
    labels = [label for i, label in enumerate(labels) if embeddings[i] is not None]
    embeddings = [emb for emb in embeddings if emb is not None]
    filepaths = [filepath for i, filepath in enumerate(filepaths) if embeddings[i] is not None]

    # TODO convert the embeddings and labels to torch tensors before putting them into the dataset
    embeddings = np.array(embeddings)
    labels = np.array(labels)
    dataset = Dataset_CLAP_Embeddings(embeddings=torch.from_numpy(embeddings), labels=torch.from_numpy(labels))
    return dataset




In [22]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split


audio_media_csv = '/home/ubuntu/alina/truemedia_dataset/audio-experimental_9.30.24/audio-experimental_9.30.24_sample.csv'
audio_media_dir = '/home/ubuntu/alina/truemedia_dataset/audio-experimental_9.30.24/audio-media'
dataset = get_tm_audio_dataset(embedding_model=clap_model, audio_media_csv=audio_media_csv, audio_media_dir=audio_media_dir)

# split dataset into training and validation dataset 
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


batch_size = 20
shuffle=True


train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=shuffle,
    pin_memory=True,  # Helps speed up data transfer to GPU
    drop_last=False   # Keep partial batches
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=shuffle,
    pin_memory=True,  # Helps speed up data transfer to GPU
    drop_last=False   # Keep partial batches
)


<built-in function id> Expecting value: line 1 column 1 (char 0)
<built-in function id> Expecting value: line 1 column 1 (char 0)
<built-in function id> Expecting value: line 1 column 1 (char 0)
<built-in function id> Expecting value: line 1 column 1 (char 0)
<built-in function id> Expecting value: line 1 column 1 (char 0)
<built-in function id> Expecting value: line 1 column 1 (char 0)
Total count unknown 4
Number of embedding errors 0


Training loop

In [ ]:
import torch.nn as nn
from tqdm import tqdm
import argparse
from typing import Dict, List, Union, Tuple
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
import wandb


def calculate_metrics(true_labels, predicted_probabilities, threshold=0.5):
    """
    This method calculates true media style metrics. 
    In true media world 1 = spoof, but aasist makes 1 = bonafide

    Args:
    true_labels (array-like): Original labels where 1 was bonafide
    predicted_probabilities (array-like): Original probabilities for being bonafied where higher values indicated bonafied
    threshold (float): Threshold for classification (default: 0.5)

    Returns:
    tuple: (f1, recall, precision, accuracy)
    """
    true_labels = np.array(true_labels)
    predicted_probabilities = np.array(predicted_probabilities)

    # Apply threshold to get predicted labels. This is the prediction where 1 = spoof 0 = bonafide
    predicted_labels = (predicted_probabilities >= threshold).astype(int)

    # # Debug prints
    # print("True labels (inverted):", true_labels_inverted[:10])
    # print("Predicted probabilities (inverted):", predicted_probabilities_inverted[:10])
    # print("Predicted labels:", predicted_labels[:10])

    f1 = f1_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)

    # # More debug information
    # print(f"Positive samples: {sum(true_labels_inverted)}")
    # print(f"Predicted positive: {sum(predicted_labels)}")
    # print(f"True Positives: {sum(true_labels_inverted & predicted_labels)}")
    return f1, recall, precision, accuracy

def epoch_loop(
    data_loader: DataLoader,
    model,
    device: torch.device,
    config: argparse.Namespace,
    loss_fxn: nn.Module,
    optimizer: Union[torch.optim.SGD, torch.optim.Adam] = None,
    scheduler: torch.optim.lr_scheduler = None,
    is_training: bool = True,  # if false, will record everything as validation
    logging: bool = True,
) -> Tuple[float, Dict[str, float]]:
    
    # weight = torch.FloatTensor([0.23, 0.77]).to(device)
    criterion = loss_fxn
    
    running_loss = 0
    num_total = 0.0
    epoch_metrics = {"f1": 0, "recall": 0, "precision": 0, "accuracy": 0}
    
    model.train(is_training)
    context = torch.enable_grad if is_training else torch.no_grad
    
    with context():
        for batch_x, batch_y in tqdm(data_loader):
            batch_size = batch_x.size(0)
            num_total += batch_size
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.view(-1).type(torch.int64).to(device)
            _, batch_out = model(batch_x)
            batch_loss = criterion(batch_out, batch_y)
            
            running_loss += batch_loss * batch_size
            
            prefix = "batch_trn" if is_training else "batch_val"
            
            predicted_probabilities = torch.softmax(batch_out, dim=1)[:, 1].detach().cpu().numpy()
            batch_y_cpu = batch_y.cpu().numpy()
            f1, recall, precision, accuracy = calculate_metrics(batch_y_cpu, predicted_probabilities)
            batch_metrics = {"f1": f1, "recall": recall, "precision": precision, "accuracy": accuracy}
            if is_training and logging:
                wandb.log({
                    f"{prefix}_loss": batch_loss,
                    f"{prefix}_f1": f1,
                    f"{prefix}_recall": recall,
                    f"{prefix}_precision": precision,
                    f"{prefix}_accuracy": accuracy,
                })
            
            for key in epoch_metrics:
                epoch_metrics[key] += batch_metrics[key]
            
            if is_training:
                optimizer.zero_grad()
                criterion(batch_out, batch_y).backward()
                optimizer.step()
                
                if config["optim_config"]["scheduler"] in ["cosine", "keras_decay"]:
                    scheduler.step()
                elif scheduler is None:
                    pass
                else:
                    raise ValueError(f"scheduler error, got:{scheduler}")

    num_batches = len(data_loader)
    for key in epoch_metrics:
        epoch_metrics[key] /= num_batches
    running_loss /= num_total
    
    prefix = "train" if is_training else "val"
    print(f"{prefix.capitalize()} Metrics - F1 Score: {epoch_metrics['f1']:.3f}, "
          f"Recall: {epoch_metrics['recall']:.3f}, Precision: {epoch_metrics['precision']:.3f}, "
          f"Accuracy: {epoch_metrics['accuracy']:.3f}")
    if logging:
        wandb.log({
            f"{prefix}_loss": running_loss,
            f"{prefix}_f1": epoch_metrics['f1'],
            f"{prefix}_recall": epoch_metrics['recall'],
            f"{prefix}_precision": epoch_metrics['precision'],
            f"{prefix}_accuracy": epoch_metrics['accuracy'],
        })
    
    return running_loss, epoch_metrics


In [ ]:
class BinaryClassifier1(nn.Module):
    def __init__(self, embedding_dim=512):
        super(BinaryClassifier1, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(embedding_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)
    
class BinaryClassifier2(nn.Module):
    def __init__(self, embedding_dim=512, hidden_dims=[256, 128]):
        super(BinaryClassifier2, self).__init__()
        
        layers = []
        input_dim = embedding_dim
        
        # Create hidden layers
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(0.2)
            ])
            input_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(hidden_dims[-1], 1))
        layers.append(nn.Sigmoid())
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)


In [ ]:
# main method
def train(model, trn_loader, val_loader, optimizer, device, config, model_save_path):
    # Training
    for epoch in range(config["num_epochs"]):
        print("training epoch{:03d}".format(epoch))
        
        train_loss, train_epoch_metrics = epoch_loop(data_loader=trn_loader, model=model, optimizer=optimizer, device=device,
                                   config=config, is_training=True)
        

        val_loss, val_epoch_metrics = epoch_loop(data_loader=val_loader, model=model, optimizer=optimizer, device=device,
                                config=config, is_training=False)
        print(val_epoch_metrics)

        if best_val_accuracy <= val_epoch_metrics['accuracy']:
            print("best model find at epoch", epoch)
            best_val_accuracy = val_epoch_metrics['accuracy']
            
        
        print("DONE.\nTrain Loss:{:.5f}".format(
            train_loss))
        print("")

        torch.save(model.state_dict(),
        model_save_path / "epoch_{}{}.pth".format(epoch, 
                                                    f"_{val_epoch_metrics['accuracy']:03.3f}" if val_epoch_metrics and 'accuracy' in val_epoch_metrics else ""))
    
    print("Models saved at", model_save_path)

